# Search-Engine-from-Scratch

# NAME(s) and NIA(s) 

**Author :** Stephen Appiah.
         Umar Muhammad.
**NIA :** 206637.
      208244.
    

In [1]:
import warnings
warnings.filterwarnings('ignore')

#### Load Python packages
Let's first import all the packages that you will need during this assignment.

In [3]:
# if you do not have 'nltk', the following command should work "python -m pip install nltk"
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\work\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\work\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
from collections import defaultdict
from array import array
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import math
import numpy as np
import collections
import time
from numpy import linalg as la
from nltk import word_tokenize

#### Load data into memory
Import the 500 Wikipedia articles (one article per line). For each article we have the document id, document title and document body separated by "|" character.

In [5]:
#Read JSON ata set
def read_data(docs_path):
  import json
  # Opening JSON file
  f = open(docs_path)
  # returns JSON object as
  # a dictionary
  data = json.load(f)
  f.close()
  return data

data = read_data('inputs/dataset_tweets_WHO.json')

In [6]:
#Data Display
data.get('1')


{'created_at': 'Wed Oct 13 08:46:17 +0000 2021',
 'id': 1448208458604584960,
 'id_str': '1448208458604584960',
 'full_text': '#COVID19 has shown how health emergencies and disasters affect entire communities – especially those with weak health systems, and vulnerable populations like migrants, indigenous peoples, and those living in fragile humanitarian conditions. https://t.co/jpUQpnu0V1',
 'truncated': False,
 'display_text_range': [0, 241],
 'entities': {'hashtags': [{'text': 'COVID19', 'indices': [0, 8]}],
  'symbols': [],
  'user_mentions': [],
  'urls': [],
  'media': [{'id': 1448208218463875077,
    'id_str': '1448208218463875077',
    'indices': [242, 265],
    'media_url': 'http://pbs.twimg.com/media/FBkRs6ZXMAUmafc.jpg',
    'media_url_https': 'https://pbs.twimg.com/media/FBkRs6ZXMAUmafc.jpg',
    'url': 'https://t.co/jpUQpnu0V1',
    'display_url': 'pic.twitter.com/jpUQpnu0V1',
    'expanded_url': 'https://twitter.com/WHO/status/1448208458604584960/photo/1',
    'type': 'pho

In [7]:
print("Total number of Tweets in the corpus: {}".format(len(data)))

Total number of Tweets in the corpus: 2399


# Building the Index


Processing the data

Define functions to clean the text (tokenization / bigram/ lowercase /remove punctuation /stemming)

In [8]:
stemmer = PorterStemmer()
stop_words = set(stopwords.words("english"))

def remove_stopwords(words):
    return [w for w in words if w.lower() not in stop_words]

def stem_each(words):
    return [stemmer.stem(w) for w in words]

def remove_white_space(text):
    return ' '.join(text.split())

In [9]:
def preprocess(text):
  """
  Preprocess the article text (title + body) removing stop words, stemming,
  transforming in lowercase and return the tokens of the text.

  Argument:
  line -- string (text) to be preprocessed

  Returns:
  line - a list of tokens corresponding to the input text after the preprocessing
  """
  stemmer = PorterStemmer()
  stop_words = set(stopwords.words("english"))
  text=  text.lower()
  text = text.replace('#', '')
  text=  text.split()
  text= remove_stopwords(text)
  text= stem_each(text)
  return text

In [10]:
#Eample
tweet = data.get('0')
print('full_text', tweet.get('full_text'))
print()
#phrase = "Smart-assistants like Apple’s Siri and Amazon’s Alexa recognize patterns in speech thanks to voice recognition, then infer meaning and provide a useful response."
print(preprocess(tweet.get('full_text')))

full_text It's International Day for Disaster Risk Reduction

#OpenWHO has launched a multi-tiered core curriculum to help equip you with the competencies needed to work within public health emergency response.

Start learning today &amp; be #Ready4Response:
👉 https://t.co/hBFFOF0xKL https://t.co/fgZY22RWuS

['intern', 'day', 'disast', 'risk', 'reduct', 'openwho', 'launch', 'multi-ti', 'core', 'curriculum', 'help', 'equip', 'compet', 'need', 'work', 'within', 'public', 'health', 'emerg', 'response.', 'start', 'learn', 'today', '&amp;', 'ready4response:', '👉', 'https://t.co/hbffof0xkl', 'https://t.co/fgzy22rwu']


#### Inverted Index Construction

So the first step in building a text search engine is assembling an inverted index. An inverted index is a data structure that maps tokens to the documents they appear in. In this context, we can consider a token to simply be a word, so an inverted index, at its most basic, is just something that takes in a word, and returns to us a list of the documents it appears in.

In [11]:
def inverted_index(data, num_documents):
    """
    Implement the inverted index and compute tf, df and idf
    
    Argument:
    data -- collection of tweets
    num_documents -- total number of documents
    
    Returns:
    index - the inverted index (implemented through a Python dictionary) containing terms as keys and the corresponding
    list of document these keys appears in (and the positions) as values.
    tf - normalized term frequency for each term in each document
    df - number of documents each term appear in
    idf - inverse document frequency of each term
    """
    index = defaultdict(list)
    tf = defaultdict(list)
    df = defaultdict(int)
    title_index = defaultdict(str)
    idf = defaultdict(float)
    
    for i in range(len(data)):
        tweet = data.get(f'{i}')
        page_id = i
        terms = preprocess(''.join(tweet.get('full_text'))) 
        title = tweet.get('full_text')
        title_index[page_id]=title

        ## As suggested in class, we are using a current_page_index ==> { ‘term1’: [current_doc, [list of positions]], ...,‘term_n’: [current_doc, [list of positions]]}
        ## current_page_index ==> { ‘web’: [1, [0]], ‘retrieval’: [1, [1,4]], ‘information’: [1, [2]]}
        current_page_index = {}
        for position, term in enumerate(terms):
            try:
                current_page_index[term][page_id].append(position)
            except:
                current_page_index[term]=[page_id, array('I',[position])] #'I' indicates unsigned int (int in Python)
            
        # normalized the term frequencies
        norm = 0
        for term, posting in current_page_index.items():
            norm += len(posting[1]) ** 2
        norm = math.sqrt(norm)
        
        #calculate the tf(dividing the term frequency by the above computed norm) and df weights
        for term, posting in current_page_index.items():
            tf[term].append(np.round(len(posting[1])/norm,4))
            df[term] = df[term] + 1 
            
        #merge the current page index with the main index
        for term_page, posting_page in current_page_index.items():
            index[term_page].append(posting_page)
        
        # Compute IDF
        for term in df:
            idf[term] = np.round(np.log(float(num_documents/df[term])), 4)
            
    return index, tf, df, idf, title_index

In [12]:
num_documents = len(data)
index, tf, df, idf, title_index = inverted_index(data, num_documents)

In [13]:
#Eample
print("First 5 Index results for the term 'covid19': \n{}".format(index['covid19'][:5]))

First 5 Index results for the term 'covid19': 
[[1, array('I', [0])], [6, array('I', [8])], [8, array('I', [2])], [11, array('I', [7])], [13, array('I', [8])]]


# Querying the Index


#### Ranking

When searching in a search engine, we are interested in obtain the results sorted by relevance or by some other criteria.

In [14]:
def rank_documents(terms, docs, index, idf, tf, title_index):
    """
    Perform the ranking of the results of a search based on the tf-idf weights
    
    Argument:
    terms -- list of query terms
    docs -- list of documents, to rank, matching the query
    index -- inverted index data structure
    idf -- inverted document frequencies
    tf -- term frequencies
    title_index -- mapping between page id and page title
    
    Returns:
    Print the list of ranked documents
    """
    
    doc_vectors = defaultdict(lambda: [0] * len(terms))
    query_vector = [0] * len(terms)
    query_terms_count = collections.Counter(terms)
    query_norm = la.norm(list(query_terms_count.values()))
    for termIndex, term in enumerate(terms):
        if term not in index: continue
        query_vector[termIndex]= query_terms_count[term]/query_norm * idf[term]
        for doc_index, (doc, postings) in enumerate(index[term]):
            if doc in docs:
                doc_vectors[doc][termIndex] = tf[term][doc_index] * idf[term]  # TODO: check if multiply for idf
    
    doc_scores=[[np.dot(curDocVec, query_vector), doc] for doc, curDocVec in doc_vectors.items() ]
    doc_scores.sort(reverse=True)
    result_docs = [x[1] for x in doc_scores]
    if len(result_docs) == 0: 
        print("No results found, try again")
        query = input()
        docs = search(query, index)
    return result_docs

#### Search

Enter queries to retrieved the top 10 relevant links based on the cosine similarity score (ltc.ltn schema).

**This search uses TF-IDF + cosine similarity**

In [15]:
def search(query, index):
    """
    output is the list of documents that contain any of the query terms. 
    So, we will get the list of documents for each query term, and take the union of them.
    """
    query = preprocess(query)
    docs = set()
    for term in query:
        try:                      
            term_docs=[posting[0] for posting in index[term]]
            docs |= set(term_docs)
        except:
            pass
    docs = list(docs)
    ranked_docs = rank_documents(query, docs, index, idf, tf, title_index)
    return ranked_docs

In [16]:
print("Insert your query (i.e.: covid19):\n")
query = input()
docs = search(query.lower(), index)
top = 10

print("\n======================\nSample of {} results out of {} for the searched query:\n".format(top, len(docs)))
for d_id in docs[:top]:
  tweet = data.get(f'{d_id}')
  if tweet:
    user = tweet.get('user')
    entities = tweet.get('entities')
    hashtags = entities.get('hashtags')
    title = title_index[d_id]
    date = tweet.get('created_at')
    like = tweet.get('like')
    retweets = tweet.get('retweet_count')
    url = tweet.get('url')
    # if hashtags:
    #   title = hashtags[0]['text']
    print("Tweet_id = {} | Title = {} | Username: {} | Date: {} | Hashtags: {} | Likes: {} | Retweets: {} | Url: {}".format(d_id, title, user, date, hashtags, like, retweets, url))

Insert your query (i.e.: covid19):

covid19

Sample of 10 results out of 604 for the searched query:

Tweet_id = 1820 | Title = When will the #COVID19 pandemic be over? https://t.co/bebc6ccur1 | Username: {'id': 14499829, 'id_str': '14499829', 'name': 'World Health Organization (WHO)', 'screen_name': 'WHO', 'location': 'Geneva, Switzerland', 'description': 'We are the #UnitedNations’ health agency - #HealthForAll.\n▶️ Always check our latest tweets on #COVID19 for updated advice/information.', 'url': 'https://t.co/wVulKuROWG', 'entities': {'url': {'urls': [{'url': 'https://t.co/wVulKuROWG', 'expanded_url': 'http://www.who.int', 'display_url': 'who.int', 'indices': [0, 23]}]}, 'description': {'urls': []}}, 'protected': False, 'followers_count': 9963586, 'friends_count': 1743, 'listed_count': 34215, 'created_at': 'Wed Apr 23 19:56:27 +0000 2008', 'favourites_count': 11879, 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'verified': True, 'statuses_count': 64983, 'lang': None,

# **Ranking and Search Variant**

#### Ranking

When searching in a search engine, we are interested in obtain the results sorted by Retweets

In [18]:
def rank_documents_by_retweets(query, docs):
    """
    Perform the ranking of the results of a search based on the Retweets
    
    Argument:
    query -- list of query terms
    docs -- list of documents, to rank, matching the query
    
    Returns:
    Print the list of ranked documents
    """
    terms = preprocess(query)
    new_data_set = []
    for term in terms:
      for index in range(len(docs)):
        tweet = data.get(f'{index}')
        full_text = tweet.get('full_text')
        full_text = preprocess(full_text)
        if term in full_text:
          new_data_set.append(tweet)
  

    result_docs = sorted(new_data_set, key=lambda x: x.get('retweet_count'), reverse=True)

    if len(result_docs) == 0: 
        print("No results found, try again")
        query = input()
        docs = search(query, index)
    return result_docs

Search

Enter queries to retrieved the top 10 relevant links based on the cosine similarity score Retweets + cosine similarity

**This search uses Retweets + cosine similarity**

In [19]:
def search_variant(query, index):
    """
    output is the list of documents that contain any of the query terms. 
    So, we will get the list of documents for each query term, and take the union of them.
    """
    ranked_docs = rank_documents_by_retweets(query, index)
    return ranked_docs

In [20]:
print("Insert your query (i.e.: covid19):\n")
query = input()
tweets = search_variant(query.lower(), data)
top = 10

Insert your query (i.e.: covid19):

covid19


In [21]:
print("\n======================\nSample of {} results out of {} for the searched query:\n".format(top, len(tweets)))
i = 0
for tweet in tweets:
  if i == top: break
  title = tweet.get('full_text')
  retweets = tweet.get('retweet_count')
  i = i+1

  print("Title = {}| Retweets: {}".format( title, retweets))


Sample of 10 results out of 604 for the searched query:

Title = RT @DrTedros: Inspirational news from #BTSARMY in #Indonesia 🇮🇩, who came together for @BTS_twt &amp; organized a #COVID19 vaccination drive fo…| Retweets: 20366
Title = RT @WHO: The #COVID19 situation globally is very dangerous with high levels of transmission driven by 4️⃣ major factors:
1️⃣ virus variants…| Retweets: 4219
Title = The #COVID19 situation globally is very dangerous with high levels of transmission driven by 4️⃣ major factors:
1️⃣ virus variants
2️⃣ increased social mixing and mobility of people
3️⃣ inappropriate use of public health &amp; social measures
4️⃣ vaccine inequity 
- Dr @mvankerkhove https://t.co/BKBuTXHbBU| Retweets: 4219
Title = A second dose of Pfizer or Moderna can be used after a first dose of AstraZeneca in some situations. #COVID19 https://t.co/dErbKOrny0| Retweets: 3151
Title = RT @DrTedros: The time is indeed now to do it right because #COVID19 is not over until it’s over:
#WearAMask


# Search using Word2Vec

The idea is to create a new word2vec representation of our dataset.
When a query is made, we create a word2vec version of our query. Then cosine similarity between our query vector with all vectors of our dataset. Finally, we sort by the cosine similarity. Hence we get the most relevant document first.

**Load Model**

In [32]:
import gensim.downloader as api
from gensim.models import Word2Vec


Load model

In [33]:
dataset = api.load("text8")  # load dataset as iterable
word2vec = Word2Vec(dataset)  # train w2v model

Create the mean vector  and define the cosine similarity function

In [34]:
from sklearn.metrics.pairwise import cosine_similarity as cs

def cosine_similarity(a, b):
  cos_sim_scikit = cs([a], [b])
  return cos_sim_scikit

def has_representation(sentence):
    '''
        Check whether the sentence has vector representation.
        Check if at least one word of the sentence exists in the language model.
    '''
    try:
      words = word_tokenize(sentence)
      return len([word for word in words if word in word2vec]) > 0
    except e:
      return 0


# mean avg of word vectors
def mean_words_vector(sentence):
    '''
        Compute the mean of words vector
    '''
    words = word_tokenize(sentence)
    words = [word for word in words if has_representation(word)]
    try:
      if len(words) == 0:
          words = ['unk']
      else:
        return np.mean(word2vec[words], axis=0)
    except:
      words = ['unk']

    return words
    

**Example Word2Vec**

In [35]:
tweet = data.get(f'0')
full_text = tweet.get('full_text')
full_text_vec = mean_words_vector(full_text)
print('Tweet Title:', full_text)
print(full_text_vec)

NameError: name 'e' is not defined

Convert our dataser of tweets into a data set representing each tweet in word2vec

In [ ]:
def data_vec_representation(data):
  """
  Return a new data set, where each tweet as a word2vec representation.
  We only store the full text representation.
  """
  new_data_set = []
  for index in range(len(data)):
    tweet = data.get(f'{index}')
    full_text = tweet.get('full_text')
    query = preprocess(full_text)
    full_text = ' '.join(query)
    full_text_vec = mean_words_vector(full_text)
    new_data_set.append((tweet, full_text_vec))
  
  return new_data_set

Rank document by cosine similarity as param : tuple (value, cosine similarity value)

In [ ]:
def rank_documents_word2vec(data):
  """
  Rank element by the most similar items first
  """
  newlist = sorted(data, key=lambda tup: tup[1], reverse=True)
  return newlist

In [29]:
def searchWord2Vec(query, data_2_vec):
    """
    Search using word2vec + cosine similarity.
    """
    data = []
    query = preprocess(query)
    term = ' '.join(query)
    term_vec = mean_words_vector(term)

    if 'unk' in term_vec:
      print("Unknown term ", term)
      return
    
    for item in data_2_vec:
      tweet , tweet_vec = item
      if 'unk' in tweet_vec:
        continue

      similarity = cosine_similarity(term_vec, tweet_vec)
      res = (tweet, similarity)
      data.append(res)

    ranked_docs = rank_documents_word2vec(data)
    return ranked_docs

Test code

In [28]:
data_set_word_2_vec = data_vec_representation(data)

NameError: name 'data_vec_representation' is not defined

In [27]:
tweet , tweet_vec = data_set_word_2_vec[0]
print(tweet_vec)

NameError: name 'data_set_word_2_vec' is not defined

In [ ]:
print("Insert your query (i.e.: How many human died today):\n")
query = input()
documents = searchWord2Vec(query.lower(), data_set_word_2_vec)
top = 20
print("\n======================\nSample of {} results out of {} for the searched query:\n".format(top, len(documents)))
i = 0
for item in documents:
  tweet , similarity = item
  if i == top: break
  title = tweet.get('full_text')
  retweets = tweet.get('retweet_count')
  i = i+1
  print("Title = {}| Retweets: {} | Similarity {} ".format( title, retweets, similarity))
  print("\n")


**Return a top-20 list of documents for each of the 5 queries, using word2vec + cosine similarity.**

In [ ]:
def queryTest(query):
  documents = searchWord2Vec(query, data_set_word_2_vec)
  top = 20
  print("\n======================\nSample of {} results out of {} for the searched query:\n".format(top, len(documents)))
  i = 0
  for item in documents:
    tweet , similarity = item
    if i == top: break
    title = tweet.get('full_text')
    retweets = tweet.get('retweet_count')
    i = i+1
    print("Title = {}| Retweets: {} | Similarity {} ".format( title, retweets, similarity))
    print("\n")

In [ ]:
queryTest("Human")


Sample of 20 results out of 2384 for the searched query:

Title = @DrTedros "Urbanization, deforestation, #climatechange and intensified agricultural practices are all increasing the risks of zooneses spilling over into human populations"-@DrTedros| Retweets: 30 | Similarity [[0.6684346]] 


Title = For all women, the 🆕 #CervicalCancer guidelines now recommend the HPV DNA 🧬 test as the primary method for cervical screening.
This detects the high-risk strains of the human papillomavirus (HPV) that cause almost all cervical cancers.
 
👉 https://t.co/hd1NNcbKP2 https://t.co/0fbqdewNHo| Retweets: 77 | Similarity [[0.63172054]] 


Title = It's #WorldHumanitarianDay
 
The health argument for climate action is clear. The same unsustainable choices that destroy our planet are affecting our health. Safeguarding human health from #ClimateChange impacts 🌪️☀️🌊🔥  is more urgent than ever.

#TheHumanRace https://t.co/3EpeK20gIN| Retweets: 969 | Similarity [[0.6245643]] 


Title = @Olympics @DrTedro

In [ ]:
queryTest("How many human died today")


Sample of 20 results out of 2384 for the searched query:

Title = Today is #WorldMosquitoDay 🦟!

1 child 🧒 dies every 2 minutes ⏰ from mosquito-borne malaria.

Sleeping 💤 under insecticide-treated nets is one of the most effective ways to prevent #malaria. https://t.co/kC3uNHhuZG| Retweets: 495 | Similarity [[0.58923846]] 


Title = Today is #InternationalSafeAbortionDay. Access to safe #abortion protects the health and promotes the human rights of women and girls

https://t.co/LBCUghiBh2 https://t.co/II1HRd6KjK| Retweets: 397 | Similarity [[0.5583078]] 


Title = Today is International Day of Older Persons and a reminder that people of all ages play a valuable role in our society.
 
Stand up against ageism and help us create #AWorld4AllAges 

👉 https://t.co/citXXbrRLE https://t.co/x5i8UhU7Bl| Retweets: 76 | Similarity [[0.5554859]] 


Title = @DrTedros @UN "We accept that we should have taken stronger measures to screen candidates and ensure more effective human resources processes"-

In [ ]:
queryTest("Food that affect human health")


Sample of 20 results out of 2384 for the searched query:

Title = Oral diseases - incl. dental caries, gum disease and oral cancer - affect an estimated 3.5 billion people globally. 
They affect health, well-being and quality of life, especially where resources for prevention, diagnosis and treatment are limited. 

More: https://t.co/DagJPV2jdi https://t.co/2q7XEtHhHf| Retweets: 187 | Similarity [[0.8218045]] 


Title = #ClimateCrisis harms our health!
#ClimateCrisis harms our health!
#ClimateCrisis harms our health!

Limiting global warming to 1.5°C isn't only the right thing to do but also a shared responsibility for health.

🆕 WHO #COP26 Special Report explains why 👉https://t.co/WXMdMgPSWv https://t.co/r8NFs1oI37| Retweets: 481 | Similarity [[0.8112449]] 


Title = It's #WorldHumanitarianDay
 
The health argument for climate action is clear. The same unsustainable choices that destroy our planet are affecting our health. Safeguarding human health from #ClimateChange impacts 🌪️☀️🌊🔥 

In [ ]:
queryTest("Stay at home")


Sample of 20 results out of 2384 for the searched query:

Title = @Olympics @DrTedros @Tokyo2020 @Paralympics @g20org @ACTAccelerator "Every day, the choices we make as individuals can be the difference between life and death, like staying at home, keeping your distance from others, wearing a mask, opening a window and cleaning your hands"-@DrTedros #Tokyo2020 #AGoal4All https://t.co/DIaj2o4M9u| Retweets: 22 | Similarity [[0.6918276]] 


Title = Hospitals can support mothers to breastfeed by:
✅ letting mothers and babies stay together day and night
✅ making sure that mothers of sick babies can stay near their babies.

Support #breastfeeding mothers! 
🤱🏻
🤱🏼
🤱🏽
🤱🏾
🤱🏿
#WorldBreastfeedingWeek https://t.co/CsfHK6RMDY| Retweets: 55 | Similarity [[0.6884181]] 


Title = If you are experiencing violence at home during and need to leave in a hurry:
🏘️ Identify a friend, neighbour, relative or shelter you can go to
🔀 Plan how to get there
🎒 Keep ready essential personal items to take with you

